In [23]:
from dask.distributed import Client, LocalCluster
#client = Client(timeout=90)
#client = Client(processes = False, timeout=90)
c = LocalCluster(n_workers=10, threads_per_worker = 4)
client = Client(c,timeout=90)

C:\Users\jminucci\AppData\Local\Continuum\anaconda3\envs\pyabc\lib\site-packages\distributed\bokeh\core.py:56: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [4]:
from VarroaPy.VarroaPy.RunVarroaPop import VarroaPop
from timeit import default_timer as timer
import math
import time

In [5]:
def one_vp(x):
    params = {"ICWorkerAdults": 10000}
    vp = VarroaPop(params, verbose = False, save = True)
    vp.run_model()
    output = vp.get_output()
    return output

In [6]:
def batch_vp(x):
    for i in range(0,30):
        params = {"ICWorkerAdults": 10000}
        vp = VarroaPop(params, verbose = False, save = True)
        vp.run_model()
        output = vp.get_output()
    return output

In [12]:
def lg_batch_vp(x):
    for i in range(0,100):
        params = {"ICWorkerAdults": 10000}
        vp = VarroaPop(params, verbose = False, save = True)
        vp.run_model()
        output = vp.get_output()
    return output

In [89]:
#in series = 66.2 sec for 30
start = timer()
batch_vp(1)
end=timer()
print(end - start)

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

In [10]:
# 1 process 40 threads = 40.9 s
# 40 processes 1 thread each = sometimes giving weird errors, may be faster w less processes + threads
# 5 processes 8 threads each = 26 s
# 10 processes 4 threads each = 20 s
# 20 processes 2 threads each = 25 s
start = timer()
result = client.map(one_vp, range(0,50), pure = False)
client.gather(result)
end=timer()
print(end - start)

20.615306144306032


Strategy when running many batches of many runs

In [25]:
#parallelize each batch, sequential inside batch, 4000 total

#1 process 40 threads: 1562 s = 26 mins = 0.39 s/VP run
#10 processes, 4 threads each: 1570 s = 26 mins 

start = timer()
result = client.map(lg_batch_vp, range(0,40), pure = False)
client.gather(result)
end=timer()
print(end - start)

1570.621620114025


In [18]:
#Parallel inside a batch, sequential outside, 4000 total

#1 process 40 threads: 1958 = 33 mins = 0.49 s/VP run

start = timer()
for i in range(0,40):
    result = client.map(one_vp, range(0,100), pure = False)
    client.gather(result)
end=timer()
print(end - start)

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exeD:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exeD:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exeD:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exeD:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exeD:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe


D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exeD:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neoni

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

...

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exeD:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neonic_abc\VarroaPy\VarroaPy\files/exe/VarroaPop.exe
D:\Git_files\bee_neo

In [7]:
def sqrt(x):
    return math.sqrt(x)

def slow(x):
    time.sleep(5)
    return x

In [11]:
start = timer()
result = client.map(slow, range(0,45), pure = False)
client.gather(result)
end=timer()
print(end - start)

26.73243610614287


In [81]:
start = timer()
for i in range(0,20):
    slow(i)
end=timer()
print(end - start)

100.30051711273609


In [24]:
client

Client Scheduler: tcp://127.0.0.1:54094 Dashboard: http://127.0.0.1:54097/status,Cluster Workers: 10 Cores: 40 Memory: 34.28 GB


In [26]:
client.close()